In [1]:
!pip install datetime
!pip install numpy
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
# For getting weekday from date
import datetime

In [3]:
df = pd.read_csv('agg3.csv')
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
print(df.head())

   Unnamed: 0 Timestamp_dt                                 TxnID  \
0           0   2025-01-01  532ceea0-846f-4689-80e7-953913c8cbd5   
1           1   2025-01-01  cdaeae81-0137-426a-92c7-c88726184fa7   
2           2   2025-01-01  68cfb36d-e2a9-4f70-ad4b-3f0029cd17cb   
3           3   2025-01-01  a5c24ed3-55d7-487a-8de9-cd2e95235a1b   
4           4   2025-01-01  e4d77002-c783-4785-9bdc-e6bf19c3144c   

     Timestamp_x  Online CustomerID SrcAccount DstAccount Channel  \
0  2025/1/1 0:00       0    C000352  A92e83abd        NaN     ATM   
1  2025/1/1 0:02       0    C000965  A3f209383        NaN     ATM   
2  2025/1/1 0:03       0    C000441  Adb76359d        NaN     ATM   
3  2025/1/1 0:04       0    C000582  A4ae7d26c        NaN     ATM   
4  2025/1/1 0:05       0    C000973  A5b5e1a2c        NaN     ATM   

        MCC_Group  ...  DeviceID  Actions Action_Count Login_Count  \
0       Groceries  ...       NaN      NaN            0           0   
1  Salary/Benefit  ...       NaN    

In [4]:
# Confirmed: we have account data on all destination accounts
for account in df['DstAccount']:
    if pd.isna(account):
        pass
    else:
        if (not(accounts['AccountID'] == account).any()):
            print("uh oh")

In [5]:
## is the transaction in the user's home city
df['in_home_city'] = (df['HomeCity'] == df['City'])
# check is True
print(df['in_home_city'][28])
## TODO: How should we deal with the value for in person transactions? Default to 0 or 1?
#df.loc[idx, 'in_home_city'] = np.nan

True


In [6]:
## how many times have they previously made a transaction in this city
df['Timestamp_x'] = pd.to_datetime(df['Timestamp_x'], format='ISO8601')
df = df.sort_values('Timestamp_x')
df['prev_trans_in_city'] = df.groupby(['City', 'CustomerID']).cumcount()

In [7]:
## How many times has the user previously used this device
df['times_device_used'] = df.groupby(['CustomerID', 'DeviceID']).cumcount()

In [9]:
df = df.sort_values(['CustomerID', 'Timestamp_x'])

def cumulative_unique_devices(g):
    seen = set()
    counts = []
    for device in g['DeviceID']:
        counts.append(len(seen))
        seen.add(device)
    return pd.Series(counts, index=g.index)

df['unique_devices_past'] = df.groupby('CustomerID', group_keys=False).apply(cumulative_unique_devices, include_groups=False)
idx = df['Online'] == 0
df.loc[idx, 'unique_devices_past'] = np.nan

df

,Unnamed: 0,Timestamp_dt,TxnID,Timestamp_x,Online,CustomerID,SrcAccount,DstAccount,Channel,MCC_Group,...,Logout_Count,Account_View_Count,Payment_Count,Transfer_Count,Day_of_Week,Timestamp,in_home_city,prev_trans_in_city,times_device_used,unique_devices_past
1967,1967,2025-01-03,da3b1da1-2e6f-430b-ab02-b1678bbfccb8,2025-01-03 09:39:00,0,C000005,A166e5515,NaN,branch,Online Retail,...,0,0,0,0,Friday,09:39:00,False,NaN,NaN,NaN
2624,2624,2025-01-04,19307e39-75f2-44ea-a1ce-6b6cc2f51f2e,2025-01-04 03:14:00,0,C000005,A166e5515,NaN,branch,Other,...,0,0,0,0,Saturday,03:14:00,False,NaN,NaN,NaN
4867,4867,2025-01-06,aa2c3179-2917-4398-86ec-0f2e079f1128,2025-01-06 19:32:00,0,C000005,A166e5515,NaN,branch,Restaurants,...,0,0,0,0,Monday,19:32:00,False,NaN,NaN,NaN
6036,6036,2025-01-08,dfd36246-ff9a-4458-98dc-df89aaa15cf2,2025-01-08 04:54:44,1,C000005,A166e5515,NaN,online,Online Retail,...,1,0,1,0,Wednesday,04:54:00,True,0.0,0.0,1.0
6701,6701,2025-01-09,c4439812-4659-48fc-8b9b-2b0d2e6009d7,2025-01-09 00:16:00,0,C000005,A166e5515,NaN,ATM,Restaurants,...,0,0,0,0,Thursday,00:16:00,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150502,150502,2025-06-28,d5e6c023-54ab-4f31-ae8a-c3264b6a6302,2025-06-28 19:01:00,0,C001000,Aeab65b63,NaN,ATM,Other,...,0,0,0,0,Saturday,19:01:00,False,NaN,NaN,NaN
150669,150669,2025-06-28,197a9dfb-e09d-4921-94b5-3928646baa63,2025-06-28 23:19:00,0,C001000,Aeab65b63,NaN,branch,Groceries,...,0,0,0,0,Saturday,23:19:00,False,NaN,NaN,NaN
150827,150827,2025-06-29,8ee2858f-8d36-498d-a235-a4d31481be86,2025-06-29 03:56:00,0,C001000,A5cd91130,NaN,branch,Utilities,...,0,0,0,0,Sunday,03:56:00,False,NaN,NaN,NaN
150849,150849,2025-06-29,f96e1705-41a3-43cb-8bb7-e8e3e488451a,2025-06-29 04:27:00,0,C001000,A5cd91130,NaN,branch,Other,...,0,0,0,0,Sunday,04:27:00,False,NaN,NaN,NaN
